Faça o upload de um arquivo .zip com os seus dados e use o comando abaixo após a finalização do upload para a extração do conteúdo do arquivo no colab.

Implemente as funções a seguir.

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\moura\anaconda3\envs\visao\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\moura\anaconda3\envs\visao\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\moura\anaconda3\envs\visao\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Us

In [2]:

class CarSimDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path) 
        label = self.img_labels.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)  
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label


Crie as variáveis a seguir a partir da classe implementada acima.

In [3]:
from torchvision import transforms

# Ajuste nas transformações para assegurar que o tensor seja convertido para float, normalizado e com aumento de dados aplicado
transform = transforms.Compose([
    transforms.Resize((28, 28)),  
    transforms.RandomRotation(degrees=5),  
    transforms.ColorJitter(brightness=0.2), 
    transforms.RandomHorizontalFlip(),  
    transforms.ConvertImageDtype(torch.float), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

# Transformação dos rótulos (normalizando os valores de ângulo, se aplicável)
def normalize_labels(label):
    max_angle = 25  
    return label / max_angle

# Dataset de treino com transformações atualizadas
train_dataset = CarSimDataset(
    annotations_file='C:\\Users\\moura\\simulator\\dados_2\\driving_log.csv',
    img_dir='C:\\Users\\moura\\simulator\\dados_2\\IMG',
    transform=transform,
    target_transform=normalize_labels 
)

# Dataset de teste
test_dataset = CarSimDataset(
    annotations_file='C:\\Users\\moura\\simulator\\dados_2\\driving_log.csv',
    img_dir='C:\\Users\\moura\\simulator\\dados_2\\IMG',
    transform=transform,
    target_transform=normalize_labels
)


In [57]:
train_dataset

O código abaixo será utilizado para treinar o modelo com o seu dataset.

In [4]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 28, 28])
Shape of y: torch.Size([64]) torch.float32


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits.squeeze()

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2352, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [7]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.008030  [   64/ 3434]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [66]:
torch.save(model.state_dict(), "model_0.2.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Faça o download do seu modelo após o treinamento, caso queira testá-lo no simulador.

O código a seguir demonstra como o modelo será usado para inferência no simulador. Caso seja necessário, altere a função *preprocess*.

In [ ]:
def preprocess(x):
    # TODO: se necessário, alterar função
    return x

In [ ]:
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

FileNotFoundError: [Errno 2] No such file or directory: 'model_0.1.pth'

In [ ]:
model.eval()
x = test_data[0][0]
with torch.no_grad():
    x = preprocess(x)
    x = x.to(device)
    pred = model(x)

NameError: name 'test_data' is not defined